In [1]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
import torch.nn as nn
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
import torch
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
from util.node import Node
import shutil
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
import pickle
from util.func import get_patch_size
import random
from util.data import ModifiedLabelLoader
from tqdm import tqdm

Heatmaps showing where a prototype is found will not be generated because OpenCV is not installed.


In [2]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/010-CUB-27-imgnet_OOD_cnext26_img=224_nprotos=20'
# run_path = '/home/harishbabu/projects/PIPNet/runs/031-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/032-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/035-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/043-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = "/home/harishbabu/projects/PIPNet/runs/036-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel_uniformity"
# run_path = "/home/harishbabu/projects/PIPNet/runs/041-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/042-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"
run_path = "/home/harishbabu/projects/PIPNet/runs/044-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20-or-4per-desc_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/046-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=10per-desc_orth-on-rel"
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('/home/harishbabu/data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

------------------------- No discretization -------------------------


In [3]:
print(root)

root
	052+053
		cub_052_Pied_billed_Grebe
		053+050
			cub_053_Western_Grebe
			050+051
				cub_050_Eared_Grebe
				cub_051_Horned_Grebe
	004+086
		004+032
			cub_004_Groove_billed_Ani
			032+033
				cub_032_Mangrove_Cuckoo
				033+031
					cub_033_Yellow_billed_Cuckoo
					cub_031_Black_billed_Cuckoo
		086+045
			cub_086_Pacific_Loon
			045+101
				045+003
					cub_045_Northern_Fulmar
					003+002
						cub_003_Sooty_Albatross
						002+001
							cub_002_Laysan_Albatross
							cub_001_Black_footed_Albatross
				101+023
					101+100
						cub_101_White_Pelican
						cub_100_Brown_Pelican
					023+025
						cub_023_Brandt_Cormorant
						025+024
							cub_025_Pelagic_Cormorant
							cub_024_Red_faced_Cormorant



In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
net.eval()
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
# with torch.no_grad():
#     xs1, _, _ = next(iter(trainloader))
#     xs1 = xs1.to(device)
#     proto_features, _, _ = net(xs1)
#     wshape = proto_features['root'].shape[-1]
#     args.wshape = wshape #needed for calculating image patch size
#     print("Output shape: ", proto_features['root'].shape, flush=True)
    
args.wshape = 26

Num classes (k) =  18 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_023_Brandt_Cormorant'] etc.
Classes:  {'cub_001_Black_footed_Albatross': 0, 'cub_002_Laysan_Albatross': 1, 'cub_003_Sooty_Albatross': 2, 'cub_004_Groove_billed_Ani': 3, 'cub_023_Brandt_Cormorant': 4, 'cub_024_Red_faced_Cormorant': 5, 'cub_025_Pelagic_Cormorant': 6, 'cub_031_Black_billed_Cuckoo': 7, 'cub_032_Mangrove_Cuckoo': 8, 'cub_033_Yellow_billed_Cuckoo': 9, 'cub_045_Northern_Fulmar': 10, 'cub_050_Eared_Grebe': 11, 'cub_051_Horned_Grebe': 12, 'cub_052_Pied_billed_Grebe': 13, 'cub_053_Western_Grebe': 14, 'cub_086_Pacific_Loon': 15, 'cub_100_Brown_Pelican': 16, 'cub_101_White_Pelican': 17}


/home/harishbabu/.conda/envs/hpnet1/lib/python3.8/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Number of prototypes:  20


AttributeError: 'Namespace' object has no attribute 'num_protos_per_descendant'

In [5]:
optimizer_net, optimizer_classifier, params_to_freeze, params_to_train, params_backbone = get_optimizer_nn(net, args)
scheduler_net = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, T_max=len(trainloader_pretraining)*args.epochs_pretrain, eta_min=args.lr_block/100., last_epoch=-1)

if args.epochs<=30:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=5, eta_min=0.001, T_mult=1, verbose=False)
else:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=10, eta_min=0.001, T_mult=1, verbose=False)
finetune = False

if args.OOD_dataset:
    trainloader_OOD, trainloader_pretraining_OOD, trainloader_normal_OOD, trainloader_normal_augment_OOD, projectloader_OOD, testloader_OOD, test_projectloader_OOD, _ = get_dataloaders(args, device, OOD=True)
    print('-'*25 + 'Using OOD data' + '-'*25)
else:
    trainloader_OOD = trainloader_pretraining_OOD = trainloader_normal_OOD = trainloader_normal_augment_OOD = projectloader_OOD = testloader_OOD = test_projectloader_OOD = None
    print('-'*25 + 'Not using OOD data' + '-'*25)

test_info, log_dict = test_pipnet(net, testloader, optimizer_net, optimizer_classifier, \
                                  scheduler_net, scheduler_classifier, criterion, epoch, \
                                    args.epochs, device, pretrain=False, finetune=finetune, \
                                    test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                        wandb_run=None, pretrain_epochs=args.epochs_pretrain, log=None, wandb_logging=False)

chosen network is convnext
Num classes (k) =  172 ['cub_005_Crested_Auklet', 'cub_006_Least_Auklet', 'cub_007_Parakeet_Auklet', 'cub_008_Rhinoceros_Auklet', 'cub_009_Brewer_Blackbird'] etc.
-------------------------Using OOD data-------------------------


Test Epochlast: 100% 8/8 [00:13<00:00,  1.72s/it, L:8.440,LC:0.118, LA:0.02, LT:0.012, L_OOD:0.738, L_ORTH:0.001]

	Fine accuracy: 0.84
	Node name: root, acc: 99.02, f1:99.02, samples: 1022, 052+053=236/240=0.98, 004+086=776/782=0.99
	Node name: 052+053, acc: 99.17, f1:99.17, samples: 240, cub_052_Pied_billed_Grebe=60/60=1.0, 053+050=178/180=0.99
	Node name: 004+086, acc: 98.47, f1:98.47, samples: 782, 004+032=218/224=0.97, 086+045=552/558=0.99
	Node name: 053+050, acc: 98.89, f1:98.88, samples: 180, cub_053_Western_Grebe=58/60=0.97, 050+051=120/120=1.0
	Node name: 004+032, acc: 94.64, f1:94.78, samples: 224, cub_004_Groove_billed_Ani=60/60=1.0, 032+033=152/164=0.93
	Node name: 086+045, acc: 97.85, f1:97.85, samples: 558, cub_086_Pacific_Loon=54/60=0.9, 045+101=492/498=0.99
	Node name: 050+051, acc: 83.33, f1:83.33, samples: 120, cub_050_Eared_Grebe=50/60=0.83, cub_051_Horned_Grebe=50/60=0.83
	Node name: 032+033, acc: 92.68, f1:92.77, samples: 164, cub_032_Mangrove_Cuckoo=42/46=0.91, 033+031=110/118=0.93
	Node name: 045+101, acc: 98.39, f1:98.39, samples: 498, 045+003=230/236=0.97, 101+023=260/262=

In [7]:
# Proto activations on leaf descendents - topk images (prunes irrelevant protos based on threshold)

from util.data import ModifiedLabelLoader
from collections import defaultdict
import heapq
import pdb
from util.vis_pipnet import get_img_coordinates
import torchvision.transforms as transforms
from PIL import Image, ImageDraw as D
import torchvision

topk = 10
save_images = True
THRESHOLD = 0.5

def get_heap():
    list_ = []
    heapq.heapify(list_)
    return list_

patchsize, skip = get_patch_size(args)

for node in root.nodes_with_children():
#     if node.name == 'root':
#         continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

    name2label = projectloader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(projectloader, node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}
    
    imgs = modifiedLabelLoader.filtered_imgs

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> list of top-k activations
    proto_mean_activations = defaultdict(lambda: defaultdict(get_heap))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
        if coarse_label2name[ys.item()] not in non_leaf_children_names:
            continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            softmaxes, pooled, _ = net(xs, inference=False)
            pooled = pooled[node.name].squeeze(0) 
            softmaxes = softmaxes[node.name]#.squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]
                
                # Take the max per prototype.                             
                max_per_prototype, max_idx_per_prototype = torch.max(softmaxes, dim=0)
                max_per_prototype_h, max_idx_per_prototype_h = torch.max(max_per_prototype, dim=1)
                max_per_prototype_w, max_idx_per_prototype_w = torch.max(max_per_prototype_h, dim=1) #shape (num_prototypes)
                
                h_idx = max_idx_per_prototype_h[p, max_idx_per_prototype_w[p]]
                w_idx = max_idx_per_prototype_w[p]

                if len(relevant_proto_class_names) == 0:
                    continue
                
                if (len(relevant_proto_class_names) == 1) and (relevant_proto_class_names[0] not in non_leaf_children_names):
                    continue
                    
                h_coor_min, h_coor_max, w_coor_min, w_coor_max = get_img_coordinates(args.image_size, softmaxes.shape, patchsize, skip, h_idx, w_idx)
                
                if (coarse_label2name[ys.item()] in relevant_proto_class_names):
                    child_node = root.get_node(coarse_label2name[ys.item()])
                    leaf_descendent = label2name[orig_y.item()][4:7]
                    img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                    if topk and (len(proto_mean_activations[p][leaf_descendent]) > topk):
                        heapq.heappushpop(proto_mean_activations[p][leaf_descendent], (pooled[p].item(), img_to_open, (h_coor_min, h_coor_max, w_coor_min, w_coor_max)))
                    else:
                        heapq.heappush(proto_mean_activations[p][leaf_descendent], (pooled[p].item(), img_to_open, (h_coor_min, h_coor_max, w_coor_min, w_coor_max)))

                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)

    
    print('Node', node.name)
    # NOTE: Prototypes are assumed to be exclusive to only one child class, may not always be the case
    for child_classname in class_and_prototypes:
        
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            topk_mean_activations = []
            logstr = '\t'*2 + f'Proto:{p} '
            for leaf_descendent in proto_mean_activations[p]:
                mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                topk_mean_activations.append(mean_activation)
                num_images = len(proto_mean_activations[p][leaf_descendent])
                logstr += f'{leaf_descendent}:({mean_activation}) '
            
            relevant_proto = np.all(np.array(topk_mean_activations) > THRESHOLD)
            
            # pruning - setting weight of overspecific prototypes to zero
            if not relevant_proto:
                classification_layer = getattr(net.module, '_'+node.name+'_classification')
                torch.nn.init.zeros_(classification_layer.weight[:,p])
            
            
            print(('SHARED ' if relevant_proto else 'OVERSPECIFIC ') + logstr)
            
print('Done!!')

Node root
	 Child: 052+053
OVERSPECIFIC 		Proto:0 050:(0.3723) 051:(0.7658) 052:(0.0123) 053:(0.0155) 
OVERSPECIFIC 		Proto:128 050:(0.2763) 051:(0.2181) 052:(0.0174) 053:(0.3347) 
OVERSPECIFIC 		Proto:4 050:(0.2687) 051:(0.5082) 052:(0.0011) 053:(0.611) 
OVERSPECIFIC 		Proto:7 050:(0.1752) 051:(0.7296) 052:(0.0082) 053:(0.5884) 
OVERSPECIFIC 		Proto:15 050:(0.5338) 051:(0.0025) 052:(0.8759) 053:(0.0795) 
OVERSPECIFIC 		Proto:34 050:(0.2063) 051:(0.3641) 052:(0.0377) 053:(0.7544) 
OVERSPECIFIC 		Proto:36 050:(0.0453) 051:(0.6006) 052:(0.0024) 053:(0.9617) 
OVERSPECIFIC 		Proto:42 050:(0.4489) 051:(0.4824) 052:(0.7552) 053:(0.0216) 
OVERSPECIFIC 		Proto:43 050:(0.0287) 051:(0.0283) 052:(0.0284) 053:(0.9671) 
OVERSPECIFIC 		Proto:44 050:(0.0125) 051:(0.0113) 052:(0.9011) 053:(0.0253) 
OVERSPECIFIC 		Proto:49 050:(0.3413) 051:(0.862) 052:(0.0011) 053:(0.8667) 
OVERSPECIFIC 		Proto:70 050:(0.0916) 051:(0.6271) 052:(0.003) 053:(0.9891) 
OVERSPECIFIC 		Proto:72 050:(0.6494) 051:(0.3755) 052:

Node 052+053
	 Child: 053+050
SHARED 		Proto:3 050:(0.7745) 051:(0.8589) 053:(0.9539) 
OVERSPECIFIC 		Proto:4 050:(0.4065) 051:(0.8929) 053:(0.9978) 
OVERSPECIFIC 		Proto:9 050:(0.7372) 051:(0.9643) 053:(0.4975) 
SHARED 		Proto:12 050:(0.8935) 051:(0.7556) 053:(0.9578) 
SHARED 		Proto:13 050:(0.796) 051:(0.8591) 053:(0.8746) 
SHARED 		Proto:16 050:(0.7435) 051:(0.8951) 053:(0.8183) 
OVERSPECIFIC 		Proto:17 050:(0.1952) 051:(0.4513) 053:(0.9874) 
OVERSPECIFIC 		Proto:19 050:(0.4399) 051:(0.9609) 053:(0.8205) 
SHARED 		Proto:21 050:(0.6943) 051:(0.9255) 053:(0.9824) 


Node 004+086
	 Child: 086+045
OVERSPECIFIC 		Proto:0 001:(0.7419) 002:(0.8428) 003:(0.0661) 023:(0.0023) 024:(0.0022) 025:(0.0023) 045:(0.8792) 086:(0.0028) 100:(0.0023) 101:(0.0041) 
OVERSPECIFIC 		Proto:1 001:(0.0394) 002:(0.0155) 003:(0.063) 023:(0.9403) 024:(0.8906) 025:(0.997) 045:(0.0051) 086:(0.5115) 100:(0.2262) 101:(0.0401) 
OVERSPECIFIC 		Proto:5 001:(0.3691) 002:(0.724) 003:(0.6323) 023:(0.0441) 024:(0.0777) 025:(0.0075) 045:(0.002) 086:(0.0542) 100:(0.8595) 101:(0.5693) 
OVERSPECIFIC 		Proto:10 001:(0.3339) 002:(0.4376) 003:(0.3306) 023:(0.6726) 024:(0.0813) 025:(0.3724) 045:(0.0378) 086:(0.3658) 100:(0.0471) 101:(0.0018) 
OVERSPECIFIC 		Proto:11 001:(0.3273) 002:(0.0041) 003:(0.0136) 023:(0.9386) 024:(0.0702) 025:(0.3309) 045:(0.0354) 086:(0.5285) 100:(0.0091) 101:(0.0015) 
OVERSPECIFIC 		Proto:17 001:(0.9544) 002:(0.9707) 003:(0.2135) 023:(0.1767) 024:(0.0125) 025:(0.0049) 045:(0.0402) 086:(0.1349) 100:(0.9247) 101:(0.9685) 
OVERSPECIFIC 		Proto:18 001:(0.521) 002:(0.3962

Node 053+050
	 Child: 050+051
OVERSPECIFIC 		Proto:0 050:(0.4518) 051:(0.6865) 
SHARED 		Proto:3 050:(0.8887) 051:(0.9972) 
OVERSPECIFIC 		Proto:4 050:(0.262) 051:(0.7437) 
SHARED 		Proto:6 050:(0.9723) 051:(0.9659) 
SHARED 		Proto:8 050:(0.9204) 051:(0.9509) 
OVERSPECIFIC 		Proto:16 050:(0.3581) 051:(0.8679) 
SHARED 		Proto:18 050:(0.8088) 051:(0.9333) 
SHARED 		Proto:19 050:(0.9359) 051:(0.8258) 


Node 004+032
	 Child: 032+033
SHARED 		Proto:0 031:(0.9532) 032:(0.8901) 033:(0.9248) 
SHARED 		Proto:4 031:(0.9077) 032:(0.9834) 033:(0.8925) 
OVERSPECIFIC 		Proto:6 031:(0.2572) 032:(0.7965) 033:(0.9404) 
SHARED 		Proto:8 031:(0.8281) 032:(0.9778) 033:(0.9613) 
OVERSPECIFIC 		Proto:12 031:(0.9947) 032:(0.4127) 033:(0.9744) 
SHARED 		Proto:14 031:(0.9221) 032:(0.5728) 033:(0.8774) 
OVERSPECIFIC 		Proto:18 031:(0.9827) 032:(0.0461) 033:(0.9177) 
SHARED 		Proto:19 031:(0.9988) 032:(0.9965) 033:(0.9594) 
SHARED 		Proto:22 031:(0.5851) 032:(0.9314) 033:(0.9901) 


...

Node 045+101
	 Child: 101+023
OVERSPECIFIC 		Proto:1 023:(0.693) 024:(0.9674) 025:(0.8513) 100:(0.004) 101:(0.004) 
OVERSPECIFIC 		Proto:33 023:(0.9611) 024:(0.5683) 025:(0.9901) 100:(0.3083) 101:(0.0076) 
OVERSPECIFIC 		Proto:35 023:(0.0742) 024:(0.9828) 025:(0.0041) 100:(0.9821) 101:(0.9897) 
OVERSPECIFIC 		Proto:8 023:(0.858) 024:(0.9943) 025:(0.9897) 100:(0.0266) 101:(0.1589) 
OVERSPECIFIC 		Proto:40 023:(0.9831) 024:(0.9955) 025:(0.9921) 100:(0.7705) 101:(0.2546) 
OVERSPECIFIC 		Proto:44 023:(0.9983) 024:(0.9995) 025:(0.9996) 100:(0.002) 101:(0.0012) 
OVERSPECIFIC 		Proto:46 023:(0.5892) 024:(0.1531) 025:(0.8785) 100:(0.1764) 101:(0.9774) 
OVERSPECIFIC 		Proto:19 023:(0.9072) 024:(0.9955) 025:(0.4954) 100:(0.3348) 101:(0.0014) 
OVERSPECIFIC 		Proto:52 023:(0.088) 024:(0.019) 025:(0.0283) 100:(0.9354) 101:(0.9812) 
OVERSPECIFIC 		Proto:53 023:(0.7084) 024:(0.0273) 025:(0.1527) 100:(0.959) 101:(0.0132) 
OVERSPECIFIC 		Proto:57 023:(0.0158) 024:(0.6905) 025:(0.0083) 100:(0.926) 101:(

Node 045+003
	 Child: 003+002
SHARED 		Proto:6 001:(0.8151) 002:(0.8045) 003:(0.7592) 
SHARED 		Proto:8 001:(0.901) 002:(0.831) 003:(0.995) 
SHARED 		Proto:9 001:(0.9851) 002:(0.9979) 003:(0.988) 
SHARED 		Proto:11 001:(0.9955) 002:(0.9868) 003:(0.9836) 
SHARED 		Proto:13 001:(0.9742) 002:(0.9523) 003:(0.8751) 
SHARED 		Proto:17 001:(0.9984) 002:(0.9999) 003:(0.9919) 
OVERSPECIFIC 		Proto:21 001:(0.9942) 002:(0.0968) 003:(0.9705) 
SHARED 		Proto:23 001:(0.9219) 002:(0.9853) 003:(0.7604) 


Node 101+023
	 Child: 023+025
SHARED 		Proto:2 023:(0.988) 024:(0.9883) 025:(0.6582) 
SHARED 		Proto:8 023:(0.8023) 024:(0.9999) 025:(0.9828) 
SHARED 		Proto:9 023:(0.803) 024:(0.7869) 025:(0.773) 
SHARED 		Proto:18 023:(0.996) 024:(0.9842) 025:(0.9956) 
SHARED 		Proto:20 023:(0.8247) 024:(0.9982) 025:(0.8065) 
SHARED 		Proto:24 023:(0.9171) 024:(0.9901) 025:(0.9944) 
SHARED 		Proto:25 023:(0.992) 024:(0.9809) 025:(0.9993) 
SHARED 		Proto:30 023:(0.8427) 024:(0.9735) 025:(0.8279) 
SHARED 		Proto:31 023:(0.8521) 024:(0.9856) 025:(0.941) 
	 Child: 101+100
SHARED 		Proto:3 100:(0.9985) 101:(0.9951) 
SHARED 		Proto:6 100:(0.9984) 101:(0.9991) 
SHARED 		Proto:7 100:(0.9962) 101:(0.9986) 
SHARED 		Proto:14 100:(0.9981) 101:(0.9989) 
SHARED 		Proto:26 100:(0.9941) 101:(0.9772) 
SHARED 		Proto:28 100:(0.9937) 101:(0.9847) 


Node 003+002
	 Child: 002+001
SHARED 		Proto:0 001:(0.9997) 002:(0.9986) 
SHARED 		Proto:12 001:(0.9945) 002:(0.9844) 
SHARED 		Proto:13 001:(0.9911) 002:(0.9926) 
SHARED 		Proto:17 001:(0.9999) 002:(0.9968) 
SHARED 		Proto:18 001:(0.9724) 002:(0.9732) 


Node 023+025
	 Child: 025+024
SHARED 		Proto:3 024:(0.9715) 025:(0.983) 
SHARED 		Proto:6 024:(0.9997) 025:(0.9402) 
SHARED 		Proto:7 024:(0.9959) 025:(0.9742) 
SHARED 		Proto:9 024:(0.9898) 025:(0.9649) 
SHARED 		Proto:13 024:(0.9978) 025:(0.9899) 
SHARED 		Proto:14 024:(0.9993) 025:(0.9713) 
Done!!


In [8]:
torch.save({'model_state_dict': net.state_dict(),\
            'optimizer_net_state_dict': optimizer_net.state_dict(),\
            'optimizer_classifier_state_dict': optimizer_classifier.state_dict()},\
           os.path.join(os.path.join(args.log_dir, 'checkpoints'),\
            f'net_overspecific_pruned_thresh={THRESHOLD}_{str(epoch)}'))

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# ---------------------------------------------------------------------------------------------------------------------------
# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
pruned_net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
pruned_net = pruned_net.to(device=device)
pruned_net = nn.DataParallel(pruned_net, device_ids = device_ids)    
pruned_net.load_state_dict(checkpoint['model_state_dict'],strict=True)
pruned_net.eval()

# ---------------------------------------------------------------------------------------------------------------------------
# Create a convolutional network based on arguments and add 1x1 conv layer
# feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# # Create a PIP-Net
# replaced_net = PIPNet(num_classes=len(classes),
#                     num_prototypes=num_prototypes,
#                     feature_net = feature_net,
#                     args = args,
#                     add_on_layers = add_on_layers,
#                     pool_layer = pool_layer,
#                     classification_layers = classification_layers,
#                     num_parent_nodes = len(root.nodes_with_children()),
#                     root = root
#                     )
# replaced_net = replaced_net.to(device=device)
# replaced_net = nn.DataParallel(replaced_net, device_ids = device_ids)    
# replaced_net.load_state_dict(checkpoint['model_state_dict'],strict=True)
# replaced_net.eval()

# ---------------------------------------------------------------------------------------------------------------------------
criterion = nn.NLLLoss(reduction='mean').to(device)

args.wshape = 26


Num classes (k) =  18 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_023_Brandt_Cormorant'] etc.
Classes:  {'cub_001_Black_footed_Albatross': 0, 'cub_002_Laysan_Albatross': 1, 'cub_003_Sooty_Albatross': 2, 'cub_004_Groove_billed_Ani': 3, 'cub_023_Brandt_Cormorant': 4, 'cub_024_Red_faced_Cormorant': 5, 'cub_025_Pelagic_Cormorant': 6, 'cub_031_Black_billed_Cuckoo': 7, 'cub_032_Mangrove_Cuckoo': 8, 'cub_033_Yellow_billed_Cuckoo': 9, 'cub_045_Northern_Fulmar': 10, 'cub_050_Eared_Grebe': 11, 'cub_051_Horned_Grebe': 12, 'cub_052_Pied_billed_Grebe': 13, 'cub_053_Western_Grebe': 14, 'cub_086_Pacific_Loon': 15, 'cub_100_Brown_Pelican': 16, 'cub_101_White_Pelican': 17}
Number of prototypes:  20
----------Prototypes per descendant: 4----------
Assigned 136 protos to node root
Assigned 24 protos to node 052+053
Assigned 104 protos to node 004+086
Assigned 16 protos to node 053+050
Assigned 24 protos to node 004+032
Assign

# Replace proto with parent protos

In [38]:
pruned_checkpoint = torch.load(os.path.join(os.path.join(args.log_dir, 'checkpoints'),\
            f'net_overspecific_pruned_thresh={THRESHOLD}_{str(epoch)}'), map_location=device)

import copy

pruned_net.load_state_dict(pruned_checkpoint['model_state_dict'],strict=True)
# replaced_net.load_state_dict(pruned_checkpoint['model_state_dict'],strict=True)
replaced_net = copy.deepcopy(pruned_net)

In [39]:
import pdb
from collections import defaultdict
acc_before_after = defaultdict(lambda : {'any_relevant_proto':False, 'replaced':False, 'acc_before_replacing':None, 'acc_after_replacing':None})

# First iterate through all nodes and check which ones have relevant protos
with torch.no_grad():
    for node in root.nodes_with_children():
        if node.name == 'root':
            continue

        protos = getattr(pruned_net.module, '_'+node.name+'_add_on').weight
        mlp = getattr(pruned_net.module, '_'+node.name+'_classification').weight
        for class_idx in node.children_to_labels.values():
            relevant_proto_idxs = np.where((mlp[class_idx, :] > 1e-3).cpu())[0]
            if len(relevant_proto_idxs) > 0:
                acc_before_after[node.name]['any_relevant_proto'] = True
                # even if one of the classes of the node has relevant protos that is enough to say the node has relevant protos
                break

with torch.no_grad():
    for node in root.nodes_with_children():
        if node.name == 'root':
            continue
        if not acc_before_after[node.name]['any_relevant_proto']:
            continue

        parent_node = node.parent

        parent_protos = getattr(pruned_net.module, '_'+parent_node.name+'_add_on').weight
        parent_mlp = getattr(pruned_net.module, '_'+parent_node.name+'_classification').weight
        class_idx = parent_node.children_to_labels[node.name]
        parent_relevant_proto_idxs = np.where((parent_mlp[class_idx, :] > 1e-3).cpu())[0]

        if len(parent_relevant_proto_idxs) == 0:
            continue
        print(node.name)
        parent_relevant_protos = parent_protos[parent_relevant_proto_idxs]

        child_protos = getattr(pruned_net.module, '_'+node.name+'_add_on').weight
        child_mlp = getattr(pruned_net.module, '_'+node.name+'_classification').weight

        relevant_ps = set()
        for c in range(child_mlp.shape[0]):
            proto_weights = child_mlp[c,:]
            for p in range(child_mlp.shape[1]):
                if proto_weights[p]> 1e-3:
                    relevant_ps.add(p)

        child_relevant_proto_idxs = np.array(list(relevant_ps))

        c_idx_to_closest_p_idx = {}
        for c_idx in child_relevant_proto_idxs:
            closest_dist = float('inf')
            closest_p_idx = None
            for p_idx in parent_relevant_proto_idxs:
                inner_product = torch.sum(parent_protos[p_idx].squeeze() * child_protos[c_idx].squeeze()).item()
                if inner_product < closest_dist:
                    closest_dist = inner_product
                    closest_p_idx = p_idx
            c_idx_to_closest_p_idx[c_idx] = closest_p_idx
            getattr(replaced_net.module, '_'+node.name+'_add_on').weight[c_idx, :, :, :] = getattr(pruned_net.module, '_'+parent_node.name+'_add_on').weight[closest_p_idx, :, :, :]
            
        acc_before_after[node.name]['replaced'] = True

053+050
050+051
032+033
033+031
045+003
003+002
101+100
023+025
002+001
025+024


In [43]:
nodes_with_relevant_protos = set([node_name for node_name, details in acc_before_after.items() \
                              if details['any_relevant_proto']])

nodes_with_replaced_protos = set([node_name for node_name, details in acc_before_after.items() \
                              if details['replaced']])

In [44]:
print(nodes_with_relevant_protos)
print(nodes_with_replaced_protos)

{'101+023', '032+033', '033+031', '045+101', '003+002', '101+100', '052+053', '004+032', '045+003', '023+025', '053+050', '025+024', '002+001', '050+051', '086+045'}
{'032+033', '033+031', '003+002', '101+100', '045+003', '023+025', '053+050', '025+024', '002+001', '050+051'}


# Custom test funtion that skips nodes with no relevant proto

In [45]:
from tqdm import tqdm
import torch
import torch.nn.functional as F
import torch.optim
import torch.utils.data
import math
import numpy as np

from collections import defaultdict
from torchmetrics.functional import f1_score, recall, precision
from torchvision.datasets.folder import ImageFolder
from pipnet.train import uniform_loss, align_loss, orth_dist

OOD_LABEL = -1

def test_pruned_pipnet(net, nodes_with_relevant_protos, test_loader, optimizer_net, optimizer_classifier, scheduler_net, scheduler_classifier, criterion, epoch, nr_epochs, device, pretrain=False, finetune=False, progress_prefix: str = 'Test Epoch', wandb_logging=True, test_loader_OOD=None, kernel_orth=False, wandb_run=None, pretrain_epochs=0, log:Log=None):

    root = net.module.root
    dataset = test_loader.dataset
    while type(dataset) != ImageFolder:
        dataset = dataset.dataset
    name2label = dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    label2name[OOD_LABEL] = 'OOD'

    wandb_log_subdir = 'test'

    # node_accuracy = defaultdict(lambda: {'n_examples': 0, 'n_correct': 0, 'accuracy': None, 'preds': None, 'children': defaultdict(lambda: {'n_examples': 0, 'n_correct': 0})})
    node_accuracy = {}
    for node in root.nodes_with_children():
        node_accuracy[node.name] = {'n_examples': 0, 'n_correct': 0, 'accuracy': None, 'f1': None, 'preds': torch.empty(0, node.num_children()).to(device), 'gts': torch.empty(0).to(device)}
        node_accuracy[node.name]['children'] = defaultdict(lambda: {'n_examples': 0, 'n_correct': 0})

    # Make sure the model is in eval mode
    net.eval()
    
    # Store info about the procedure
    test_info = dict()
    total_loss = 0.
    total_acc = 0.
    class_loss_ep_mean = 0.
    a_loss_pf_ep_mean = 0.
    tanh_loss_ep_mean = 0.
    OOD_loss_ep_mean = 0.
    kernel_orth_loss_ep_mean = 0.
    uni_loss_ep_mean = 0.

    iters = len(test_loader)
    # Show progress on progress bar. 
    test_iter = tqdm(enumerate(test_loader),
                    total=len(test_loader),
                    desc=progress_prefix+'%s'%epoch,
                    mininterval=2.,
                    ncols=0)
    
    test_OOD_iter = iter(test_loader_OOD) if test_loader_OOD else None
    OOD_loss_required = True if test_loader_OOD else False

    if pretrain:
        align_pf_weight = (epoch/nr_epochs)*1.
        unif_weight = 0.5 #ignored
        t_weight = 5.
        cl_weight = 0.
        OOD_loss_weight = 0.
        orth_weight = 0.5
    else:
        align_pf_weight = 5. 
        t_weight = 2.
        unif_weight = 0.
        cl_weight = 2.
        OOD_loss_weight = 0.2
        orth_weight = 0.5

    # maps, node_name -> loss_name -> list_of_loss_values_corresponding_to_each_step
    # node_wise_losses = defaultdict(lambda: defaultdict(list))
    node_wise_losses = {}
    for node in root.nodes_with_children():
        node_wise_losses[node.name] = {}
        node_wise_losses[node.name]['class_loss'] = []
        node_wise_losses[node.name]['a_loss'] = []
        node_wise_losses[node.name]['tanh_loss'] = []
        node_wise_losses[node.name]['OOD_loss'] = []
        node_wise_losses[node.name]['kernel_orth_loss'] = []
        node_wise_losses[node.name]['uni_loss'] = []

    
    lrs_net = []
    lrs_class = []
    n_fine_correct = 0
    n_samples = 0
    
    with torch.no_grad():
        # Iterate through the data set to update leaves, prototypes and network
        for i, (xs, ys) in test_iter:       
            
            xs, ys = xs.to(device), ys.to(device)

            if test_OOD_iter:
                xs_OOD, ys_OOD = next(test_OOD_iter)
                ys_OOD.fill_(OOD_LABEL)
                xs_OOD, ys_OOD = xs_OOD.to(device), ys_OOD.to(device)
                xs = torch.cat([xs, xs, xs_OOD])
                ys = torch.cat([ys, ys, ys_OOD])
            else:
                xs = torch.cat([xs, xs])
                ys = torch.cat([ys, ys])
            
            # Perform a forward pass through the network
            proto_features, pooled, out = net(xs)
            
            loss, class_loss_dict, a_loss_pf_dict, tanh_loss_dict, OOD_loss_dict, kernel_orth_loss_dict, uni_loss_dict, avg_class_loss, avg_a_loss_pf, avg_tanh_loss, avg_OOD_loss, avg_kernel_orth_loss, avg_uni_loss, acc = \
            calculate_loss_pruned_net(net, nodes_with_relevant_protos, proto_features, pooled, out, ys, align_pf_weight, t_weight, unif_weight, cl_weight, OOD_loss_weight, orth_weight, net.module._multiplier, pretrain, finetune, \
                           criterion, test_iter, print=True, EPS=1e-8, root=root, label2name=label2name, node_accuracy=node_accuracy, OOD_loss_required=OOD_loss_required, kernel_orth=kernel_orth)
            
            for node_name, loss_value in class_loss_dict.items():
                if node_name not in nodes_with_relevant_protos:
                    continue
                node_wise_losses[node_name]['class_loss'].append(loss_value.item())
            
            for node_name, loss_value in a_loss_pf_dict.items():
                if node_name not in nodes_with_relevant_protos:
                    continue
                node_wise_losses[node_name]['a_loss'].append(loss_value.item())

            for node_name, loss_value in tanh_loss_dict.items():
                if node_name not in nodes_with_relevant_protos:
                    continue
                node_wise_losses[node_name]['tanh_loss'].append(loss_value.item())

            for node_name, loss_value in OOD_loss_dict.items():
                if node_name not in nodes_with_relevant_protos:
                    continue
                node_wise_losses[node_name]['OOD_loss'].append(loss_value.item())

            for node_name, loss_value in kernel_orth_loss_dict.items():
                if node_name not in nodes_with_relevant_protos:
                    continue
                node_wise_losses[node_name]['kernel_orth_loss'].append(loss_value.item())

            for node_name, loss_value in uni_loss_dict.items():
                if node_name not in nodes_with_relevant_protos:
                    continue
                node_wise_losses[node_name]['uni_loss'].append(loss_value.item())

            class_loss_ep_mean += avg_class_loss if avg_class_loss else 0.
            a_loss_pf_ep_mean += avg_a_loss_pf if avg_a_loss_pf else 0.
            tanh_loss_ep_mean += avg_tanh_loss if avg_tanh_loss else 0.
            OOD_loss_ep_mean += avg_OOD_loss if avg_OOD_loss else 0.
            kernel_orth_loss_ep_mean += avg_kernel_orth_loss if avg_kernel_orth_loss else 0.
            uni_loss_ep_mean += avg_uni_loss if avg_uni_loss else 0.
                
            total_acc+=acc # DUMMY can be removed
            total_loss+=loss.item()
            
            _, preds_joint = net.module.get_joint_distribution(out)
            preds_joint = preds_joint[ys != OOD_LABEL]
            _, fine_predicted = torch.max(preds_joint.data, 1)
            target = ys[ys != OOD_LABEL]
            fine_correct = fine_predicted == target
            n_fine_correct += fine_correct.sum().item()
            n_samples += target.size(0)

    test_info['fine_accuracy'] = n_fine_correct/n_samples

    test_info['accuracy'] = total_acc/float(i+1)
    test_info['loss'] = total_loss/float(i+1)

    class_loss_ep_mean /= float(i+1)
    a_loss_pf_ep_mean /= float(i+1)
    tanh_loss_ep_mean /= float(i+1)
    OOD_loss_ep_mean /= float(i+1)
    kernel_orth_loss_ep_mean /= float(i+1)
    uni_loss_ep_mean /= float(i+1)

    log_dict = {}
    # if wandb_logging:
    log_dict[wandb_log_subdir + "/epoch loss"] = test_info['loss']
    log_dict[wandb_log_subdir + "/fine_accuracy"] = test_info['accuracy']
    log_dict[wandb_log_subdir + "/class_loss"] = class_loss_ep_mean
    log_dict[wandb_log_subdir + "/a_loss_pf"] = a_loss_pf_ep_mean
    log_dict[wandb_log_subdir + "/tanh_loss"] = tanh_loss_ep_mean
    log_dict[wandb_log_subdir + "/OOD_loss"] = OOD_loss_ep_mean
    log_dict[wandb_log_subdir + "/kernel_orth_loss"] = kernel_orth_loss_ep_mean
    log_dict[wandb_log_subdir + "/uni_loss"] = uni_loss_ep_mean
    # wandb_run.log({wandb_log_subdir + "/epoch loss": train_info['loss']}, step=epoch)
    # wandb_run.log({wandb_log_subdir + "/epoch lrs_net": train_info['lrs_net']})
    # wandb_run.log({wandb_log_subdir + "/epoch lrs_class": train_info['lrs_class']})

    for node_name in node_accuracy:
        if node_name not in nodes_with_relevant_protos:
            continue
        node_accuracy[node_name]['accuracy'] = round((node_accuracy[node_name]['n_correct'] / node_accuracy[node_name]['n_examples']) * 100, 2)
        node_accuracy[node_name]['f1'] = f1_score(node_accuracy[node_name]["preds"], node_accuracy[node_name]["gts"].to(torch.int), \
                                                    average='weighted', num_classes=net.module.root.get_node(node_name).num_children()).item()
        node_accuracy[node_name]['f1'] = round(node_accuracy[node_name]['f1'] * 100, 2)
        # if wandb_logging:
        log_dict[wandb_log_subdir + f"/node_wise/acc:{node_name}"] = node_accuracy[node_name]['accuracy']
        log_dict[wandb_log_subdir + f"/node_wise/f1:{node_name}"] = node_accuracy[node_name]['f1']
    if wandb_logging:
        wandb_run.log(log_dict, step=epoch if pretrain else (epoch+pretrain_epochs))

    test_info['node_accuracy'] = node_accuracy
    print('\tFine accuracy:', round(test_info['fine_accuracy'], 2))
    for node_name in node_accuracy:
        if node_name not in nodes_with_relevant_protos:
            continue
        acc = node_accuracy[node_name]["accuracy"]
        f1 = node_accuracy[node_name]['f1']
        samples = node_accuracy[node_name]["n_examples"]
        log_string = f'\tNode name: {node_name}, acc: {acc}, f1:{f1}, samples: {samples}'
        for child in net.module.root.get_node(node_name).children:
            child_n_correct = node_accuracy[node_name]['children'][child.name]['n_correct']
            child_n_examples = node_accuracy[node_name]['children'][child.name]['n_examples']
            log_string += ", " + f'{child.name}={child_n_correct}/{child_n_examples}={round(child_n_correct/child_n_examples, 2)}'
        print(log_string)

    # create a log csv file for each node to log the different loss values
    if log:
        log_sub_dir = 'node_wise_metrics_val'
        os.makedirs(os.path.join(log.log_dir, log_sub_dir), exist_ok=True)
        for node_name in node_wise_losses:
            if node_name not in nodes_with_relevant_protos:
                continue
            loss_names = sorted(list(node_wise_losses[node_name].keys()))
            try:
                log.create_log(f'{log_sub_dir}/{node_name}_losses', 'epoch', *loss_names)
            except Exception as e:
                pass

            epoch_losses = [] # contains mean over each step for each loss
            for loss_name in loss_names:
                if len(node_wise_losses[node_name][loss_name]) != 0:
                    epoch_losses.append(np.mean(node_wise_losses[node_name][loss_name]))
                else:
                    epoch_losses.append('n.a')
            log.log_values(f'{log_sub_dir}/{node_name}_losses', epoch if pretrain else (epoch+pretrain_epochs), *epoch_losses)
    
    return test_info, log_dict

def calculate_loss_pruned_net(net, nodes_with_relevant_protos, proto_features, pooled, out, ys, align_pf_weight, t_weight, unif_weight, cl_weight, OOD_loss_weight, orth_weight, net_normalization_multiplier, pretrain, finetune, criterion, train_iter, print=True, EPS=1e-10, root=None, label2name=None, node_accuracy=None, OOD_loss_required=False, kernel_orth=False):
    batch_names = [label2name[y.item()] for y in ys]
    loss = 0
    class_loss = {}
    a_loss_pf = {}
    tanh_loss = {}
    OOD_loss = {}
    kernel_orth_loss = {}
    uni_loss = {}
    for node in root.nodes_with_children():
        if node.name not in nodes_with_relevant_protos:
            continue
        children_idx = torch.tensor([name in node.descendents for name in batch_names])
        batch_names_coarsest = [node.closest_descendent_for(name).name for name in batch_names if name in node.descendents]
        node_y = torch.tensor([node.children_to_labels[name] for name in batch_names_coarsest]).cuda()

        if len(node_y) == 0:
            continue

        # ys = torch.cat([node_y,node_y])

        # pooled1, pooled2 = pooled[node.name].chunk(2)
        # pooled1 = pooled1[children_idx]
        # pooled2 = pooled2[children_idx]

        pooled1, pooled2 = pooled[node.name][children_idx].chunk(2)

        # pf1, pf2 = proto_features[node.name].chunk(2)
        # pf1 = pf1[children_idx]
        # pf2 = pf2[children_idx]

        pf1, pf2 = proto_features[node.name][children_idx].chunk(2)

        # out[node.name] = out[node.name][torch.cat([children_idx,children_idx])] # since out will have 2*batch_size samples
        # node_logits = out[node.name][torch.cat([children_idx,children_idx])] # since out will have 2*batch_size samples
        node_logits = out[node.name][children_idx]

        embv2 = pf2.flatten(start_dim=2).permute(0,2,1).flatten(end_dim=1)
        embv1 = pf1.flatten(start_dim=2).permute(0,2,1).flatten(end_dim=1)
        
        a_loss_pf[node.name] = (align_loss(embv1, embv2.detach()) \
                                + align_loss(embv2, embv1.detach())) / 2.

        tanh_loss[node.name] = -(torch.log(torch.tanh(torch.sum(pooled1,dim=0))+EPS).mean() \
                                 + torch.log(torch.tanh(torch.sum(pooled2,dim=0))+EPS).mean()) / 2.
        
        if kernel_orth:
            prototype_kernels = getattr(net.module, '_'+node.name+'_add_on')
            classification_layer = getattr(net.module, '_'+node.name+'_classification')
            # using any below because its a relevant prototype if it has strong connection to any one of the class
            relevant_prototype_kernels = prototype_kernels.weight[(classification_layer.weight > 0.001).any(dim=0)]
            kernel_orth_loss[node.name] = orth_dist(relevant_prototype_kernels)

        if not finetune:
            # pretraining or general training
            loss += align_pf_weight * a_loss_pf[node.name]
            loss += t_weight * tanh_loss[node.name]
            if kernel_orth:
                loss += orth_weight * kernel_orth_loss[node.name]
        
        if not pretrain:
            # finetuning or general training
            softmax_inputs = torch.log1p(node_logits**net_normalization_multiplier)
            class_loss[node.name] = criterion(F.log_softmax((softmax_inputs),dim=1),node_y) # * (len(node_y) / len(ys[ys != OOD_LABEL]))
            loss += cl_weight * class_loss[node.name]

            if OOD_loss_required:
                not_children_idx = torch.tensor([name not in node.descendents for name in batch_names]) # includes OOD images as well as images belonging to other nodes
                OOD_logits = out[node.name][not_children_idx] # [sum(not_children_idx), node.num_children()]
                sigmoid_out = F.sigmoid(torch.log1p(OOD_logits**net_normalization_multiplier))
                OOD_loss[node.name] = F.binary_cross_entropy(sigmoid_out, torch.zeros_like(OOD_logits))
                loss += OOD_loss_weight * OOD_loss[node.name]
        # Our tanh-loss optimizes for uniformity and was sufficient for our experiments. However, if pretraining of the prototypes is not working well for your dataset, you may try to add another uniformity loss from https://www.tongzhouwang.info/hypersphere/ Just uncomment the following three lines
        # else:
        #     uni_loss[node.name] = (uniform_loss(F.normalize(pooled1+EPS,dim=1)) + uniform_loss(F.normalize(pooled2+EPS,dim=1)))/2.
        #     loss += unif_weight * uni_loss[node.name]

        # For debugging purpose
        node_accuracy[node.name]['n_examples'] += node_y.shape[0]
        _, node_coarsest_predicted = torch.max(node_logits.data, 1)
        node_accuracy[node.name]['n_correct'] += (node_y == node_coarsest_predicted).sum().item()
        for child in node.children:
            node_accuracy[node.name]['children'][child.name]['n_examples'] += (node_y == node.children_to_labels[child.name]).sum().item()
            node_accuracy[node.name]['children'][child.name]['n_correct'] += (node_coarsest_predicted[node_y == node.children_to_labels[child.name]] == node.children_to_labels[child.name]).sum().item()
            node_accuracy[node.name]['preds'] = torch.cat((node_accuracy[node.name]['preds'], node_logits))
            node_accuracy[node.name]['gts'] = torch.cat((node_accuracy[node.name]['gts'], node_y))

    acc=0.
    # if not pretrain:
    #     ys_pred_max = torch.argmax(out, dim=1)
    #     correct = torch.sum(torch.eq(ys_pred_max, ys))
    #     acc = correct.item() / float(len(ys))
    if print: 
        with torch.no_grad():
            avg_a_loss_pf = np.mean([node_a_loss_pf.item() for node_name, node_a_loss_pf in a_loss_pf.items()])
            avg_tanh_loss = np.mean([node_tanh_loss.item() for node_name, node_tanh_loss in tanh_loss.items()])

            # optional loss, dict will be empty if not used, so setting the average to a placeholder vale
            if len(kernel_orth_loss) > 0:
                avg_kernel_orth_loss = np.mean([node_kernel_orth_loss.item() for node_name, node_kernel_orth_loss in kernel_orth_loss.items()])
            else:
                avg_kernel_orth_loss = -5 # placeholder value

            # optional loss, dict will be empty if not used, so setting the average to a placeholder vale
            if len(uni_loss) > 0:
                avg_uni_loss = np.mean([node_uni_loss.item() for node_name, node_uni_loss in uni_loss.items()])
            else:
                avg_uni_loss = -5
            
            avg_class_loss = None
            avg_OOD_loss = None
            if pretrain:
                # # optional loss, dict will be empty if not used, so setting the average to a placeholder vale
                # if len(uni_loss) > 0:
                #     avg_uni_loss = np.mean([node_uni_loss.item() for node_name, node_uni_loss in uni_loss.items()])
                # else:
                #     avg_uni_loss = -5
                train_iter.set_postfix_str(
                f'L: {loss.item():.3f}, LA:{avg_a_loss_pf.item():.2f}, LT:{avg_tanh_loss.item():.3f}, L_ORTH:{avg_kernel_orth_loss:.3f}, L_UNI:{avg_uni_loss:.3f}',refresh=False)
            else:
                avg_class_loss = np.mean([node_class_loss.item() for node_name, node_class_loss in class_loss.items()])
                avg_OOD_loss = np.mean([node_OOD_loss.item() for node_name, node_OOD_loss in OOD_loss.items()]) if OOD_loss_required else -5
                if finetune:
                    train_iter.set_postfix_str(
                    f'L:{loss.item():.3f},LC:{avg_class_loss.item():.3f}, LA:{avg_a_loss_pf.item():.2f}, LT:{avg_tanh_loss.item():.3f}, L_OOD:{avg_OOD_loss:.3f}',refresh=False)
                else:
                    train_iter.set_postfix_str(
                    f'L:{loss.item():.3f},LC:{avg_class_loss.item():.3f}, LA:{avg_a_loss_pf.item():.2f}, LT:{avg_tanh_loss.item():.3f}, L_OOD:{avg_OOD_loss:.3f}, L_ORTH:{avg_kernel_orth_loss:.3f}',refresh=False)            
    return loss, class_loss, a_loss_pf, tanh_loss, OOD_loss, kernel_orth_loss, uni_loss, avg_class_loss, avg_a_loss_pf, avg_tanh_loss, avg_OOD_loss, avg_kernel_orth_loss, avg_uni_loss, acc


# Test pruned / replaced net

In [46]:
optimizer_net, optimizer_classifier, params_to_freeze, params_to_train, params_backbone = get_optimizer_nn(net, args)
scheduler_net = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, T_max=len(trainloader_pretraining)*args.epochs_pretrain, eta_min=args.lr_block/100., last_epoch=-1)

if args.epochs<=30:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=5, eta_min=0.001, T_mult=1, verbose=False)
else:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=10, eta_min=0.001, T_mult=1, verbose=False)
finetune = False

if args.OOD_dataset:
    trainloader_OOD, trainloader_pretraining_OOD, trainloader_normal_OOD, trainloader_normal_augment_OOD, projectloader_OOD, testloader_OOD, test_projectloader_OOD, _ = get_dataloaders(args, device, OOD=True)
    print('-'*25 + 'Using OOD data' + '-'*25)
else:
    trainloader_OOD = trainloader_pretraining_OOD = trainloader_normal_OOD = trainloader_normal_augment_OOD = projectloader_OOD = testloader_OOD = test_projectloader_OOD = None
    print('-'*25 + 'Not using OOD data' + '-'*25)

test_info, pruned_log_dict = test_pruned_pipnet(pruned_net, nodes_with_relevant_protos, testloader, optimizer_net, optimizer_classifier, \
                                  scheduler_net, scheduler_classifier, criterion, epoch, \
                                    args.epochs, device, pretrain=False, finetune=finetune, \
                                    test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                        wandb_run=None, pretrain_epochs=args.epochs_pretrain, log=None, wandb_logging=False)


chosen network is convnext
Num classes (k) =  172 ['cub_005_Crested_Auklet', 'cub_006_Least_Auklet', 'cub_007_Parakeet_Auklet', 'cub_008_Rhinoceros_Auklet', 'cub_009_Brewer_Blackbird'] etc.
-------------------------Using OOD data-------------------------


Test Epochlast: 100% 8/8 [00:06<00:00,  1.29it/s, L:10.261,LC:0.213, LA:0.02, LT:0.012, L_OOD:0.734, L_ORTH:0.001]

	Fine accuracy: 0.2
	Node name: 052+053, acc: 95.0, f1:95.14, samples: 240, cub_052_Pied_billed_Grebe=60/60=1.0, 053+050=168/180=0.93
	Node name: 053+050, acc: 97.78, f1:97.78, samples: 180, cub_053_Western_Grebe=58/60=0.97, 050+051=118/120=0.98
	Node name: 004+032, acc: 94.64, f1:94.78, samples: 224, cub_004_Groove_billed_Ani=60/60=1.0, 032+033=152/164=0.93
	Node name: 086+045, acc: 10.75, f1:2.09, samples: 558, cub_086_Pacific_Loon=60/60=1.0, 045+101=0/498=0.0
	Node name: 050+051, acc: 83.33, f1:83.33, samples: 120, cub_050_Eared_Grebe=50/60=0.83, cub_051_Horned_Grebe=50/60=0.83
	Node name: 032+033, acc: 92.68, f1:92.77, samples: 164, cub_032_Mangrove_Cuckoo=42/46=0.91, 033+031=110/118=0.93
	Node name: 045+101, acc: 47.39, f1:30.47, samples: 498, 045+003=236/236=1.0, 101+023=0/262=0.0
	Node name: 033+031, acc: 88.14, f1:88.13, samples: 118, cub_033_Yellow_billed_Cuckoo=50/58=0.86, cub_031_Black_billed_Cuckoo=54/60=0.9
	Node name: 045+003, acc: 96.61, f1:96.65, samples: 236, cub_045_N

In [47]:
optimizer_net, optimizer_classifier, params_to_freeze, params_to_train, params_backbone = get_optimizer_nn(net, args)
scheduler_net = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, T_max=len(trainloader_pretraining)*args.epochs_pretrain, eta_min=args.lr_block/100., last_epoch=-1)

if args.epochs<=30:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=5, eta_min=0.001, T_mult=1, verbose=False)
else:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=10, eta_min=0.001, T_mult=1, verbose=False)
finetune = False

if args.OOD_dataset:
    trainloader_OOD, trainloader_pretraining_OOD, trainloader_normal_OOD, trainloader_normal_augment_OOD, projectloader_OOD, testloader_OOD, test_projectloader_OOD, _ = get_dataloaders(args, device, OOD=True)
    print('-'*25 + 'Using OOD data' + '-'*25)
else:
    trainloader_OOD = trainloader_pretraining_OOD = trainloader_normal_OOD = trainloader_normal_augment_OOD = projectloader_OOD = testloader_OOD = test_projectloader_OOD = None
    print('-'*25 + 'Not using OOD data' + '-'*25)

test_info, replaced_log_dict = test_pruned_pipnet(replaced_net, nodes_with_relevant_protos, testloader, optimizer_net, optimizer_classifier, \
                                  scheduler_net, scheduler_classifier, criterion, epoch, \
                                    args.epochs, device, pretrain=False, finetune=finetune, \
                                    test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                        wandb_run=None, pretrain_epochs=args.epochs_pretrain, log=None, wandb_logging=False)


chosen network is convnext
Num classes (k) =  172 ['cub_005_Crested_Auklet', 'cub_006_Least_Auklet', 'cub_007_Parakeet_Auklet', 'cub_008_Rhinoceros_Auklet', 'cub_009_Brewer_Blackbird'] etc.
-------------------------Using OOD data-------------------------


Test Epochlast: 100% 8/8 [00:06<00:00,  1.28it/s, L:35.395,LC:0.552, LA:0.02, LT:0.012, L_OOD:0.722, L_ORTH:1.957]

	Fine accuracy: 0.14
	Node name: 052+053, acc: 95.0, f1:95.14, samples: 240, cub_052_Pied_billed_Grebe=60/60=1.0, 053+050=168/180=0.93
	Node name: 053+050, acc: 67.78, f1:61.5, samples: 180, cub_053_Western_Grebe=10/60=0.17, 050+051=112/120=0.93
	Node name: 004+032, acc: 94.64, f1:94.78, samples: 224, cub_004_Groove_billed_Ani=60/60=1.0, 032+033=152/164=0.93
	Node name: 086+045, acc: 10.75, f1:2.09, samples: 558, cub_086_Pacific_Loon=60/60=1.0, 045+101=0/498=0.0
	Node name: 050+051, acc: 56.67, f1:51.25, samples: 120, cub_050_Eared_Grebe=14/60=0.23, cub_051_Horned_Grebe=54/60=0.9
	Node name: 032+033, acc: 71.95, f1:62.29, samples: 164, cub_032_Mangrove_Cuckoo=2/46=0.04, 033+031=116/118=0.98
	Node name: 045+101, acc: 47.39, f1:30.47, samples: 498, 045+003=236/236=1.0, 101+023=0/262=0.0
	Node name: 033+031, acc: 49.15, f1:47.15, samples: 118, cub_033_Yellow_billed_Cuckoo=40/58=0.69, cub_031_Black_billed_Cuckoo=18/60=0.3
	Node name: 045+003, acc: 72.88, f1:64.26, samples: 236, cub_045_Nor

In [48]:
replaced_log_dict
for node in root.nodes_with_children():
    if acc_before_after[node.name]['replaced']:
        print(node.name, 'before:', pruned_log_dict[f'test/node_wise/acc:{node.name}'], 'after:', replaced_log_dict[f'test/node_wise/acc:{node.name}'])

053+050 before: 97.78 after: 67.78
050+051 before: 83.33 after: 56.67
032+033 before: 92.68 after: 71.95
033+031 before: 88.14 after: 49.15
045+003 before: 96.61 after: 72.88
003+002 before: 90.91 after: 68.18
101+100 before: 94.0 after: 62.0
023+025 before: 83.95 after: 62.96
002+001 before: 93.33 after: 33.33
025+024 before: 92.31 after: 38.46


In [49]:
replaced_log_dict
for node in root.nodes_with_children():
    if (not acc_before_after[node.name]['replaced']) and acc_before_after[node.name]['any_relevant_proto']:
        print(node.name, 'before:', pruned_log_dict[f'test/node_wise/acc:{node.name}'], 'after:', replaced_log_dict[f'test/node_wise/acc:{node.name}'])

052+053 before: 95.0 after: 95.0
004+032 before: 94.64 after: 94.64
086+045 before: 10.75 after: 10.75
045+101 before: 47.39 after: 47.39
101+023 before: 97.71 after: 97.71


In [50]:
torch.save({'model_state_dict': replaced_net.state_dict(),\
            'optimizer_net_state_dict': optimizer_net.state_dict(),\
            'optimizer_classifier_state_dict': optimizer_classifier.state_dict()},\
           os.path.join(os.path.join(args.log_dir, 'checkpoints'),\
            f'net_overspecific_pruned_replaced_thresh={THRESHOLD}_{str(epoch)}'))